In [44]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **Agentic Rag**

**질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트**

 Agent 를 활용하여 RAG 를 수행해서 Agentic RAG 
 
 <br>

 ### **도구(tools)**

Tavily Search 는 Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용할수 있는 검색 도구입니다. <br>
검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. <br><br>
도구는 이처럼 Tavily Search 와 같은 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등이 있습니다. <br><br>

### **웹 검색도구: Tavily Search**

In [45]:
from langchain_community.tools.tavily_search import TavilySearchResults

도구 생성

In [46]:
search  = TavilySearchResults(k=6)      # 검색 결과를 6개까지 가져오겠다

`search.invoke()` 는 주어진 문자열에 대한 검색을 실행합니다.

In [47]:
# invoke() 함수에 검색하고 싶은 검색어를 넣습니다.
search.invoke("대구에 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'url': 'http://www.dealbada.com/bbs/board.php?bo_table=forum_golf&wr_id=1213729',
  'content': '폐업예정 카카오프렌즈골프 아지트 다녀온후기 ... 네이버에 전화번호 나오는데, 031-7967508 요거로나옵니다~~ 0 0 . 궁그미님의 댓글. 궁그미'},
 {'url': 'https://m.blog.naver.com/jsm12368/222621770154',
  'content': '카카오 프렌즈 샵 (신세계 8층) 대구 신세계백화점 / 대구 카카오 프렌즈샵. 영업시간. 금,토,일요일 10:30~20:30. 월~목요일 10:30~20:00. 전화'},
 {'url': 'https://www.tiendeo.co.kr/매장/대구광역시/카카오프렌즈',
  'content': '관심 있는 지점을 클릭하면 지점의 주소, 전화번호와 영업시간과 온라인에서 제공되는 모든 정보를 한눈에 볼 수 있습니다. 또한 대구광역시에서 최신 카카오프렌즈 카탈로그를 살펴보고 "블랙프렌즈데이" 기간 28/11에서~ 3/12까지 유효합니다 할인 혜택을 받으세요!'},
 {'url': 'https://www.tiendeo.co.kr/매장/중구-대구광역시/카카오프렌즈',
  'content': '중구 - 대구광역시 카카오프렌즈 영업시간,전화번호 및 주소 보기 ⌕ 원하시는 생활용품·서비스·가구 브랜드를 Tiendeo에서 확인하세요! ... 전화번호와 영업시간과 온라인에서 제공되는 모든 정보를 한눈에 볼 수 있습니다. 뉴스레터에 가입하여 중구 - 대구'},
 {'url': 'https://m.blog.naver.com/celina_827/222924602339',
  'content': '너무 귀여운 카카오 속 세상이예요ㅠㅠ\n\u200b\n경기도 성남시 분당구 판교역로 166\n💬카카오프렌즈샵 아지트점은\n판교역 1번 출구, 4번 출구를 이용하셔도 되고,\n지하통로로도 연결되어있으니 참고 하세요!\n\u200b\n💬영업시

## **Retriever**

PDF 문서를 FAISS DB 에 저장하고 조회하는 retriever 를 생성합니다.

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

In [49]:
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

In [50]:
# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [51]:
# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

In [52]:
# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings(model="text-embedding-3-small"))

In [53]:
# Retriever를 생성합니다.
retriever = vector.as_retriever()

In [54]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13462', 'producer': 'Hancom PDF 1.3.0.542', 'creationDate': "D:20231208132838+09'00'", 'modDate': "D:20231208132838+09'00'", 'trapped': ''}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 

`create_retriever_tool()` 함수로 `retriever` 를 도구로 변환합니다.

In [55]:
from langchain.tools.retriever import create_retriever_tool

In [56]:
retriever_tool = create_retriever_tool(
    retriever,              
    name="pdf_search",      # 도구의 이름을 입력합니다.
    
    # 도구에 대한 설명을 자세히 기입해야 합니다!!
    description="use this tool to search information from the PDF document", 
)

### **Agent 가 사용할 도구 목록 정의**

`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다.

In [57]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

### **Agent 생성**

Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

In [58]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [59]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)    # LLM 정의

In [60]:
prompt = ChatPromptTemplate.from_messages(              # Prompt 정의
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Tool Calling Agent 를 생성합니다.

In [61]:
from langchain.agents import create_tool_calling_agent

In [62]:
# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

agent 를 실행하는 AgentExecutor 를 생성합니다.

In [63]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

### **에이전트 실행하기**

agent_executor 객체의 invoke() 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. <br>
input 키에 사용자 질문을 값으로 할당한 딕셔너리를 인자로 전달합니다.<br>

In [64]:
from langchain_teddynote.messages import AgentStreamParser

In [65]:
# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [66]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'url': 'https://allgreat.tistory.com/321', 'content': '2024 프로야구 한국시리즈 포스트시즌 일정 진출팀 / 5위 결정전 경기 타이브레이크, 플레이오프 날짜 순위 위 프로야구 순위는 2024년 9월 29일 기준이다. 6.(일) 준플레이오프 2차전 준플레이오프(준PO) 1~2차전은 정규리그 3위 팀인 LG(엘지)의 홈구장인 잠실야구장에서 열린다. 1차전 : 2024년 10월 13일(일) 2차전 : 2024년 10월 14일(월) 3차전 : 2024년 10월 16일(수) 4차전 : 2024년 10월 17일(목) 5차전 : 2024년 10월 19일(토) 5판 3선승제로 치러지는 프로야구 플레이오프(준결승전) 1~2차전은 정규리그 2위팀인 삼성 라이온즈의 홈구장인 대구 라이온즈파크에서 열린다. 1차전 : 10월 21일(월)\xa0 3차전 : 10월 24일(목) 4차전 : 10월 25일(금) 5차전 : 10월 27일(일) 6차전 : 10월 28일(월) 7차전 : 10월 29일(화) 한국시리즈(코리아시리즈 결승전)는 7전 4선승제로 치러지며, 정규리그 1위팀 기아와 플레이오프 승리팀이 마지막 우승을 놓고 치열한 경기가 펼쳐질 예정이다.'}, {'url': 'https://m.blog.naver.com/hoga1201/223571117829', 'content': '2024년 프로야구 포스트시즌 가을야구 플레이오프 진출팀 예상 분석 상대팀 별 잔여경기 : 네이버 블로그 2024년 프로야구 포스트시즌 가을야구 플레이오프 진출팀 예상 분석 상대팀 별 잔여경기 그래서 오늘은 KBO 남은 상대팀 경기 수와 우승 및 

In [67]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 

### **이전 대화내용 기억하는 Agent**

`RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줍니다.

In [68]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}                                          # session_id 를 저장할 딕셔너리 생성

def get_session_history(session_ids):               # session_id 를 기반으로 세션 기록을 가져오는 함수
    if session_ids not in store:                    # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장

    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,    
    get_session_history,                    # 대화 session_id    
    input_messages_key="input",             # 프롬프트의 질문이 입력되는 key: "input"    
    history_messages_key="chat_history",    # 프롬프트의 메시지가 입력되는 key: "chat_history"
)

In [69]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "삼성전자가 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 

In [70]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous response into English:

- **Model Name**: Samsung Gauss
- **Announcement Date**: First unveiled on November 8, 2023, at the Samsung AI Forum 2023.
- **Components**: Samsung Gauss consists of three models: language, code, and image.
  - **Language Model**: Supports tasks such as email writing, document summarization, and translation, with various models that can operate in the cloud and on-device.
  - **Code Model**: Provides a coding assistant called 'code.i' through an interactive interface, optimized for in-house software development.
  - **Image Model**: Capable of generating creative images and modifying existing ones, as well as converting low-resolution images to high-resolution.

- **On-Device Operation**: Samsung Gauss is designed to operate on-device, ensuring that user information is not at risk of being leaked externally. This means it has been trained on secure data.

- **Future Plans**: Samsung plans to gradually integrate Sa